In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/work_space/Project/ABFR')
!pip install factor_analyzer

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from util.module import *
import torch.optim as optim
import torch.nn.functional as F

df = pd.read_csv('./Data/house-prices-advanced-regression-techniques/train.csv')
numeric_df = df.select_dtypes(include=['int', 'float'])

In [5]:
numeric_df = numeric_df.dropna(axis=0)
numeric_df.reset_index(inplace=True, drop=True)
test_df = numeric_df.iloc[:, 1:23]

array_df = np.array(test_df)
array_df = np.round(array_df, 3)

fa = Factor_attention(array_df)
fa.col_to_vec(threshold = 0.5)

In [ ]:
model = Attention(n_factor= fa.n_factors, info_dim = fa.dim_info)
model(fa)

In [6]:
model = Attention(n_factor= fa.n_factors, info_dim = fa.dim_info)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# 모델 학습
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(fa)
    loss = criterion(outputs, model(fa))
    loss.backward()
    optimizer.step()

In [10]:
class FactorAnalysis(nn.Module):
    def __init__(self, num_vars, num_factors):
        super(FactorAnalysis, self).__init__()
        self.loadings = nn.Parameter(torch.randn(num_vars, num_factors).double())
        self.psi = nn.Parameter(torch.ones(num_vars).double())  # Uniquenesses, or specific variances

    def forward(self):
        # Reconstruct covariance matrix based on FA
        return torch.mm(self.loadings, self.loadings.t()) + torch.diag(self.psi)

def em_step(model, data):
    data = data.double()
    # Covariance matrix
    S = torch.mm(data.t(), data) / data.shape[0]

    # E-step
    L_psi_inv = torch.mm(model.loadings.t(), torch.inverse(torch.diag(model.psi)))
    M = torch.inverse(torch.eye(model.loadings.shape[1]) + torch.mm(L_psi_inv, model.loadings))
    E_h = torch.mm(torch.mm(M, model.loadings.t()), torch.inverse(torch.diag(model.psi)))
    E_hhT = M + torch.mm(E_h, torch.mm(data.t(), data))

    # M-step
    model.loadings.data = torch.mm(torch.mm(S, E_h.t()), torch.inverse(E_hhT))
    psi_diag = torch.diag(S - torch.mm(model.loadings, torch.mm(E_h, S)))
    model.psi.data = psi_diag / data.shape[0]

data = model(fa)

fa_model = FactorAnalysis(data.shape[-1], data.shape[-1])

# num_epochs = 100
# for epoch in range(num_epochs):
#     em_step(fa_model, data)
#     if (epoch + 1) % 10 == 0:
#         loss = torch.norm(data - fa_model())
#         print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
class FactorAnalysis(nn.Module):
    def __init__(self, num_vars, num_factors):
        super(FactorAnalysis, self).__init__()
        self.loadings = nn.Parameter(torch.randn(num_vars, num_factors).float())
        self.psi = nn.Parameter(torch.ones(num_vars).float())  # Uniquenesses, or specific variances

    def forward(self):
        # Reconstruct covariance matrix based on FA
        return torch.mm(self.loadings, self.loadings.t()) + torch.diag(self.psi)

def em_step(model, data):
    # Ensure data is float type
    data = data.float()

    # Covariance matrix
    S = torch.mm(data.t(), data) / data.shape[0]

    # E-step
    L_psi_inv = torch.mm(model.loadings.t(), torch.inverse(torch.diag(model.psi)))
    M = torch.inverse(torch.eye(model.loadings.shape[1]) + torch.mm(L_psi_inv, model.loadings))
    E_h = torch.mm(torch.mm(M, model.loadings.t()), torch.inverse(torch.diag(model.psi)))
    E_hhT = M + torch.mm(E_h, torch.mm(data.t(), data))

    # M-step
    model.loadings.data = torch.mm(torch.mm(S, E_h.t()), torch.inverse(E_hhT))
    psi_diag = torch.diag(S - torch.mm(model.loadings, torch.mm(E_h, S)))
    model.psi.data = psi_diag / data.shape[0]

data = model(fa).float()  # Ensure data is of float type

fa_model = FactorAnalysis(data.shape[-1], data.shape[-1])

num_epochs = 100
for epoch in range(num_epochs):
    em_step(fa_model, data)
    if (epoch + 1) % 10 == 0:
        loss = torch.norm(data - fa_model())
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


RuntimeError: ignored

In [41]:
# Helper functions for EM-based FA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def e_step(X, loadings, unique_variances):
    """
    E-step of the EM algorithm for FA.
    """
    X, loadings, unique_variances = X.double(), loadings.double(), unique_variances.double()

    inv_diag_unique_variances = 1.0 / unique_variances
    inv_sigma = torch.diag(inv_diag_unique_variances) - torch.mm(
        torch.mm(loadings.t(), torch.diag(inv_diag_unique_variances)),
        torch.mm(loadings, torch.inverse(torch.eye(loadings.shape[1], device=device, dtype=torch.double) +
                                         torch.mm(loadings.t(), torch.mm(torch.diag(inv_diag_unique_variances), loadings))))
    )

    latent_mean = torch.mm(torch.mm(X, torch.diag(inv_diag_unique_variances)), torch.mm(loadings, inv_sigma))

    latent_covariance = torch.eye(loadings.shape[1], device=device, dtype=torch.double) - torch.mm(
        torch.mm(loadings.t(), torch.diag(inv_diag_unique_variances)),
        torch.mm(loadings, inv_sigma)
    ) + torch.mm(inv_sigma, torch.mm(latent_mean.t(), latent_mean))

    return latent_mean, latent_covariance

def m_step(X, latent_mean, latent_covariance):
    """
    M-step of the EM algorithm for FA.
    """
    X, latent_mean, latent_covariance = X.double(), latent_mean.double(), latent_covariance.double()

    loadings = torch.mm(torch.mm(X.t(), latent_mean), torch.inverse(latent_covariance))

    diag_residual = torch.diag(torch.mm(X.t(), X)) - torch.diag(torch.mm(loadings, torch.mm(latent_mean.t(), X)))
    unique_variances = diag_residual / X.shape[0]

    return loadings, unique_variances

def em_factor_analysis(X, n_components, max_iter=100):
    """
    Factor Analysis using the EM algorithm.
    """
    n_features = X.shape[1]

    # Initialize parameters
    loadings = torch.randn(n_features, n_components, device=device, dtype=torch.double)
    unique_variances = torch.ones(n_features, device=device, dtype=torch.double)

    for i in range(max_iter):
        # E-step
        latent_mean, latent_covariance = e_step(X, loadings, unique_variances)

        # M-step
        loadings, unique_variances = m_step(X, latent_mean, latent_covariance)

    return loadings, unique_variances

def varimax_rotation(loadings, gamma=1.0, max_iter=25, tolerance=1e-6):
    """
    Apply Varimax rotation to the loadings matrix.
    """
    loadings = loadings.double()

    n, m = loadings.shape
    rotated_loadings = loadings.clone()
    rotation_matrix = torch.eye(m, device=device, dtype=torch.double)

    for _ in range(max_iter):
        d = torch.mm(rotated_loadings, torch.mm(torch.diag(torch.sum(rotated_loadings ** 2, dim=0) ** (-gamma/2)), rotated_loadings.t()))
        u, s, v = torch.svd(torch.mm(d, rotated_loadings - (1.0 / n) * torch.mm(rotated_loadings, torch.diag(torch.sum(rotated_loadings, dim=0)))))
        rotation_matrix_new = torch.mm(u, v)
        rotated_loadings = torch.mm(loadings, rotation_matrix_new)

        if torch.norm(rotation_matrix_new - rotation_matrix) < tolerance:
            break

        rotation_matrix = rotation_matrix_new

    return rotated_loadings

def cosine_similarity_loss(loadings):
    similarity_matrix = F.cosine_similarity(loadings.unsqueeze(0), loadings.unsqueeze(1), dim=2)
    loss = torch.mean(similarity_matrix)  # 평균 Cosine Similarity
    return similarity_matrix

def cosine_similarity_matrix(tensor):
    n, m = tensor.shape
    if n != m:
        raise ValueError("The input tensor must be square (n x n).")

    result = torch.zeros((n, n))

    for i in range(n):
        for j in range(n):
            result[i][j] = F.cosine_similarity(tensor[i], tensor[j], dim=0)

    return result

# Apply EM-based FA and Varimax rotation
loadings, unique_variances = em_factor_analysis(model(fa), model(fa).shape[-1], max_iter=25)
rotated_loadings = varimax_rotation(loadings)

rotated_loadings.shape
cosine_similarity_matrix(rotated_loadings)


tensor([[ 1.0000,  0.9439, -0.1601,  0.9936,  0.8095, -0.9797,  0.9911,  0.9158,
          0.0190,  0.9236, -0.3323,  0.4235],
        [ 0.9439,  1.0000, -0.4769,  0.9751,  0.5704, -0.8590,  0.9794,  0.9970,
         -0.3119,  0.9984, -0.6238,  0.1005],
        [-0.1601, -0.4769,  1.0000, -0.2702,  0.4498, -0.0404, -0.2897, -0.5429,
          0.9838, -0.5259,  0.9826,  0.8261],
        [ 0.9936,  0.9751, -0.2702,  1.0000,  0.7383, -0.9511,  0.9998,  0.9552,
         -0.0936,  0.9610, -0.4357,  0.3186],
        [ 0.8095,  0.5704,  0.4498,  0.7383,  1.0000, -0.9105,  0.7245,  0.5058,
          0.6024,  0.5230,  0.2842,  0.8741],
        [-0.9797, -0.8590, -0.0404, -0.9511, -0.9105,  1.0000, -0.9446, -0.8171,
         -0.2186, -0.8286,  0.1365, -0.5954],
        [ 0.9911,  0.9794, -0.2897,  0.9998,  0.7245, -0.9446,  1.0000,  0.9610,
         -0.1137,  0.9664, -0.4538,  0.2993],
        [ 0.9158,  0.9970, -0.5429,  0.9552,  0.5058, -0.8171,  0.9610,  1.0000,
         -0.3839,  0.9998, -0.

In [15]:
rotated_loadings = torch.round(rotated_loadings * 1e3) / 1e3
rotated_loadings

tensor([[ 1.7742e+02, -4.9488e+02, -4.6031e+02,  5.6740e+01,  7.5120e+02,
         -2.3598e+02,  1.8599e+02, -2.8121e+02, -1.1097e+03, -9.7874e+02,
         -4.4764e+02, -1.6516e+02],
        [-6.5418e+03,  1.5266e+04,  1.4191e+04, -1.2511e+03, -2.3549e+04,
          7.4080e+03, -6.4431e+03,  1.0177e+04,  3.5688e+04,  2.9545e+04,
          1.3144e+04,  6.6798e+03],
        [ 1.5790e+01, -4.6703e+01, -4.3250e+01,  6.1210e+00,  7.0157e+01,
         -2.1784e+01,  1.6115e+01, -2.4151e+01, -1.0312e+02, -9.2683e+01,
         -4.2924e+01, -1.2877e+01],
        [-5.6695e+02,  1.2235e+03,  1.1156e+03, -1.1396e+02, -1.8599e+03,
          5.5942e+02, -4.6077e+02,  7.6539e+02,  2.8869e+03,  2.3152e+03,
          1.0361e+03,  4.6261e+02],
        [ 9.3519e+01,  9.1951e+01,  1.2387e+02, -1.0310e+01, -1.6954e+02,
          9.7813e+01, -9.9344e+01,  6.8434e+01,  9.0115e+01,  3.0623e+02,
          1.4574e+02,  8.2763e+01],
        [ 3.0886e+03, -5.6365e+03, -5.2618e+03,  1.0183e+02,  8.9873e+03,
      

In [44]:
tensor_df = torch.Tensor(array_df).double()
loadings, unique_variances = em_factor_analysis(tensor_df, tensor_df.shape[-1], max_iter=25)
rotated_loadings = varimax_rotation(loadings)

rotated_loadings.shape
cosine_similarity_matrix(rotated_loadings)

tensor([[ 1.0000,  0.9965,  0.9870,  0.9922,  0.9887,  0.9972,  0.9973,  0.9970,
          0.9887,  0.7415, -0.0455,  0.9865,  0.9925,  0.9724,  0.5537,  0.9883,
          0.8638, -0.1356,  0.9773,  0.9933,  0.9980,  0.9972],
        [ 0.9965,  1.0000,  0.9940,  0.9986,  0.9826,  0.9988,  0.9987,  0.9883,
          0.9952,  0.6963,  0.0099,  0.9955,  0.9990,  0.9756,  0.5907,  0.9945,
          0.8910, -0.1905,  0.9898,  0.9872,  0.9961,  0.9990],
        [ 0.9870,  0.9940,  1.0000,  0.9980,  0.9572,  0.9875,  0.9871,  0.9810,
          0.9960,  0.6247,  0.0821,  0.9988,  0.9972,  0.9915,  0.6740,  0.9998,
          0.9331, -0.2908,  0.9976,  0.9876,  0.9813,  0.9881],
        [ 0.9922,  0.9986,  0.9980,  1.0000,  0.9714,  0.9949,  0.9946,  0.9840,
          0.9961,  0.6581,  0.0572,  0.9991,  0.9999,  0.9831,  0.6319,  0.9983,
          0.9128, -0.2419,  0.9958,  0.9865,  0.9902,  0.9952],
        [ 0.9887,  0.9826,  0.9572,  0.9714,  1.0000,  0.9904,  0.9908,  0.9807,
          0.967

In [22]:
class TorchFactorAnalysis:
    def __init__(self):
        self.loadings = None
        self.rotation_matrix = None
        self.rotated_loadings = None
        self.factor_score = None
        self.factor_score_coefficent = None
        self.cov_mat = None
        self.explained_variance = None

    def fit(self, X, num_factor=2, rotation_type='varimax', k=4, max_iter=1000, eps=1e-05):
        assert rotation_type in ['varimax', 'promax']

        self.get_loadings(X, num_factor)
        L = self.loadings

        self.get_rotated_loadings(L, rotation_type=rotation_type,
                                  max_iter=max_iter, eps=eps, k=k)

        self.get_factor_score(X)
        return self

    def get_rotated_loadings(self, L, rotation_type, k, max_iter, eps):
        if rotation_type == 'varimax':
            R, rotated_L = self._varimax(L, max_iter=max_iter, eps=eps)
            self.rotation_matrix = R
            self.rotated_loadings = rotated_L
        else:
            R, rotated_L = self._promax(L, k=k, max_iter=max_iter, eps=eps)
            self.rotation_matrix = R
            self.rotated_loadings = rotated_L

    def _varimax(self, L, max_iter=1000, eps=1e-05):
        R = torch.eye(L.shape[1], device=device)
        d_old = 0
        p = L.shape[0]
        for i in range(max_iter):
            L_star = torch.mm(L, R)
            Q = torch.mm(L.T, L_star ** 3 - torch.mm(L_star, torch.diag(torch.diag(torch.mm(L_star.T, L_star)))/p))
            U, S, V = torch.svd(Q)
            R = torch.mm(U, V.T)
            d_new = torch.sum(S)
            if d_new < d_old * (1 + eps):
                break
            else:
                d_old = d_new
        return R, L_star

    def _promax(self, L, k=4, max_iter=1000, eps=1e-5):
        _, L_star = self._varimax(L, max_iter=max_iter, eps=eps)
        P = L_star * (torch.abs(L_star) ** (k-1))
        L_inv = torch.inverse(torch.mm(L_star.T, L_star))
        R = torch.mm(torch.mm(L_inv, L_star.T), P)
        R = R/torch.sqrt(torch.sum(R ** 2, dim=0))
        L_final = torch.mm(L_star, R)
        return R, L_final

    def get_loadings(self, X, num_factor):
        cov_mat = torch.mm((X - torch.mean(X, dim=0)).T, (X - torch.mean(X, dim=0))) / (X.shape[0]-1)
        self.cov_mat = cov_mat
        eigen_values, eigen_vectors = torch.linalg.eig(cov_mat)
        eigen_values = eigen_values[:num_factor, 0]
        eigen_vectors = eigen_vectors[:, :num_factor]
        loadings = torch.sqrt(eigen_values).unsqueeze(1) * eigen_vectors
        self.loadings = loadings
        self.explained_variance = torch.sum(eigen_values) / X.shape[1]

    def get_factor_score(self, X):
        L = self.rotated_loadings
        S_inv = torch.inverse(self.cov_mat)
        factor_score = torch.mm(torch.mm(L.T, S_inv), (X - torch.mean(X, dim=0)).T)
        self.factor_score_coefficent = (torch.mm(L.T, S_inv)).T
        self.factor_score = factor_score.T


In [23]:
temp_fa = TorchFactorAnalysis().fit(model(fa), num_factor=model(fa).shape[-1], rotation_type='promax')




IndexError: ignored